<p><img alt="Colaboratory logo" height="140px" src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" align="left" hspace="10px" vspace="0px"></p>

# **Fundamentos de programación**


El presente curso hace parte de la especialización de analítica y ciencia de datos de la facultad de Ingeniería de la Universidad de Antioquia.

## **Sesión 5**

## **Contenido**

- <a href="#gro"> GroupBy</a><br>
  - <a href="#esp"> Especificando la clave para la división del DataFrame </a><br>
  - <a href="#mul"> Objetos de múltiples índices</a><br>
  - <a href="#tab"> Tablas dinámicas</a><br>


<p><a name="esp"></a></p>


# **Especificando la clave para la división del DataFrame**

Los ejemplos presentados anteriormente expresan solo unas de las muchas opciones mediante las cuales se pueden definir los grupos. Veamos algunas otras opciones para la especificación de grupos.



La clave puede ser cualquier serie o lista con una longitud que coincida con la del DataFrame

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(data = {'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                          'data1': range(1,7),
                          'data2': np.random.randint(0, 10, 6)},
                  columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,1,3
1,B,2,9
2,C,3,9
3,A,4,8
4,B,5,5
5,C,6,7


Vamos a agrupar las filas 0 y 2 con índice "a"; las 1 y 3 con índice 1 ; las 4 y 5 con índice 10 y obtener la media

In [3]:
l = ["a", 1, "a", 1, 10, 10]

df.groupby(l).mean()

,data1,data2
1,3.0,8.5
10,5.5,6.0
a,2.0,6.0


Otro método es el de proporcionar un diccionario que asigne los valores de los índices a las claves de grupo

In [4]:
df2 = df.set_index('key').copy()
df2

,data1,data2
key,,
A,1,3
B,2,9
C,3,9
A,4,8
B,5,5
C,6,7


In [6]:
mapp = {'A': 'vocal', 'B': 'consonante', 'C': 'consonante'}

df2.groupby(mapp).sum()

,data1,data2
key,,
consonante,16,30
vocal,5,11


Análogamente al mapeo, es posible pasar cualquier función de Python que aplique sobre el índice y genere el grupo

In [8]:
type(df2.index.str)

pandas.core.strings.accessor.StringMethods

In [9]:
df2.groupby(str.lower).mean()

,data1,data2
key,,
a,2.5,5.5
b,3.5,7.0
c,4.5,8.0


<p><a name="mul"></a></p>

# **Objetos de múltiples índices**

Hasta este punto, nos hemos centrado principalmente en datos unidimensionales y bidimensionales, almacenados en Series y DataFrame, respectivamente. A menudo, es útil ir más allá y almacenar datos de mayor dimensión, es decir, datos indexados por más de una o dos claves.


Por ejemplo, cualquiera de las opciones anteriores de agrupamiento se pueden combinar para agrupar con índice múltiple

In [10]:
df = df2.groupby([str.lower, mapp]).mean()
df

,,data1,data2
key,key,,
a,vocal,2.5,5.5
b,consonante,3.5,7.0
c,consonante,4.5,8.0


Note que en este caso obtenemos un objeto con dos índices o niveles

In [11]:
df.index

MultiIndex([('a',      'vocal'),
            ('b', 'consonante'),
            ('c', 'consonante')],
           names=['key', 'key'])

Se puede acceder a los diferentes niveles del índice múltiple mediante el kwarg `level`:

In [13]:
df.groupby(level = 0).sum()

,data1,data2
key,,
a,2.5,5.5
b,3.5,7.0
c,4.5,8.0


In [14]:
df.groupby(level = 1).sum()

,data1,data2
key,,
consonante,8.0,15.0
vocal,2.5,5.5


Estudiemos este tipo de estructuras con un conjunto de datos real:

In [15]:
titanic = pd.read_csv('https://raw.githubusercontent.com/tomasate/Datos_Clases/refs/heads/main/Datos_1/titanic.csv')

In [16]:
titanic.columns = titanic.columns.str.lower()
titanic.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Veamos, por ejemplo, la tasa de supervivencia por género y por clase:

In [17]:
titanic.groupby(['sex', 'pclass']).survived.mean()

sex     pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: survived, dtype: float64

In [18]:
titanic.groupby(['sex', 'pclass'])[['survived']].mean()

survived
sex    pclass          
female 1       0.968085
       2       0.921053
       3       0.500000
male   1       0.368852
       2       0.157407
       3       0.135447

Incluyamos ahora otra agregación:

In [20]:
df = titanic.groupby(['sex', 'pclass']).survived.agg(['mean', len])
df

mean  len
sex    pclass               
female 1       0.968085   94
       2       0.921053   76
       3       0.500000  144
male   1       0.368852  122
       2       0.157407  108
       3       0.135447  347

si quisieramos obtener el número de personas que sobrevivieron podremos, por ejemplo, realizar una operación vectorizada que, como ya vimos, preserva el índice:

In [22]:
df['len'] * df['mean']

sex     pclass
female  1         91.0
        2         70.0
        3         72.0
male    1         45.0
        2         17.0
        3         47.0
dtype: float64

Con estos objetos de índice múltiple, los datos de dimensiones superiores se pueden representar de forma compacta dentro de los objetos DataFrame bidimensionales y Series unidimensionales familiares.


<p><a name="tab"></a></p>

# **Tablas dinámicas**

Volvamos a obtener la tasa de supervivencia por género y por clase

In [23]:
df = titanic.groupby(['sex', 'pclass']).survived.mean()
df

sex     pclass
female  1         0.968085
        2         0.921053
        3         0.500000
male    1         0.368852
        2         0.157407
        3         0.135447
Name: survived, dtype: float64

siempre podremos representar esta información en un DataFrame con índice único mediante el método `unstack()`:

In [24]:
df = df.unstack()
df

pclass,1,2,3
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Este GroupBy bidimensional es lo suficientemente común como para que Pandas incluya una función conveniente `pivot_table` que maneja este tipo de agregación multidimensional:

In [25]:
titanic.pivot_table('survived', index = 'sex', columns = 'pclass')

pclass,1,2,3
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Por defecto `pivot_table` aplica la función `mean()`. Para cambiar la función de agregación utilizamos el argumento `aggfunc`

In [26]:
titanic.pivot_table('survived', index = 'sex', columns = 'pclass', aggfunc = 'sum')

pclass,1,2,3
sex,,,
female,91,70,72
male,45,17,47


Podemos agrupar por dos columnas y aplicar una función correspondiente a cada una mediante un mapeo:

In [27]:
titanic.pivot_table(index = 'sex', 
                    columns = 'pclass', 
                    aggfunc = {'survived':'sum',
                               'fare': 'mean'})

fare                       survived        
pclass           1          2          3        1   2   3
sex                                                      
female  106.125798  21.970121  16.118810       91  70  72
male     67.226127  19.741782  12.661633       45  17  47

La agrupación en tablas dinámicas se puede especificar con múltiples niveles. Podríamos estar interesados en ver la edad como una tercera dimensión. Seccionaremos la edad usando la función `pd.cut`

In [28]:
pd.cut(titanic['age'], [0, 18, 80])

0      (18.0, 80.0]
1      (18.0, 80.0]
2      (18.0, 80.0]
3      (18.0, 80.0]
4      (18.0, 80.0]
           ...     
886    (18.0, 80.0]
887    (18.0, 80.0]
888             NaN
889    (18.0, 80.0]
890    (18.0, 80.0]
Name: age, Length: 891, dtype: category
Categories (2, interval[int64, right]): [(0, 18] < (18, 80]]

In [30]:
titanic.head()

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [29]:
age = pd.cut(titanic['age'], [0, 18, 80])

titanic.pivot_table('survived', index = ['sex', age], columns = 'pclass')

pclass                  1         2         3
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

Podemos aplicar esta misma estrategia para trabajar con las columnas. Agreguemos información sobre la tarifa pagada usando `pd.cut` para calcular automáticamente los cuantiles

In [32]:
fare = pd.cut(titanic['fare'], 2)

titanic.pivot_table('survived', index = ['sex', age], columns = [fare, 'pclass'])

fare            (-0.512, 256.165]                     (256.165, 512.329]
pclass                          1         2         3                  1
sex    age                                                              
female (0, 18]           0.900000  1.000000  0.511628                1.0
       (18, 80]          0.971429  0.900000  0.423729                1.0
male   (0, 18]           0.800000  0.600000  0.215686                NaN
       (18, 80]          0.369565  0.071429  0.133663                0.5

A veces es útil calcular totales a lo largo de cada grupo. Esto se puede hacer a través del kwarg `margins`

In [33]:
titanic.pivot_table('survived', index = 'sex', columns = 'pclass', aggfunc = 'sum', margins = True)

pclass,1,2,3,All
sex,,,,
female,91,70,72,233
male,45,17,47,109
All,136,87,119,342
